In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

I'm working with some data sets with a lot of columns, so I added this command to reveal all of them.

In [ ]:
pd.set_option('display.max_columns', None)

Here I imported the data and assigned it names. I also had to set the dtype for the ZIP column in the charger dataset as a string so I could later use those ZIP codes to filter chargers down to the city alone.

In [ ]:
ny_chargers = pd.read_csv('../data/Electric_Vehicle_Charging_Stations_in_New_York.csv', dtype={"ZIP": str})
nyc_cars = pd.read_csv('../data/Vehicle__Snowmobile__and_Boat_Registrations_20241203.csv')

I am checking for the formatting for the columns just to ensure I'm writing the next line peroperly.

In [ ]:
nyc_cars.head()

Now I'm writing a mass filter to get only the passenger vehicles from the five counties that make up NYC that are eletric.

In [ ]:
nyc_evs_filtered = nyc_cars[
    #This line filtered out other vehicle types like boats and snow mobiles.
    (nyc_cars['Record Type'] == 'VEH') &
    #This filtered the vehicles down to passenger vehicles, cutting out things like semis, box trucks, etc.
    (nyc_cars['Registration Class'] == 'PAS') &
    #This line filtered the entries by county of vehicle use to NYC. 
    (nyc_cars['County'].isin(['NEW YORK', 'KINGS', 'QUEENS', 'BRONX', 'RICHMOND'])) &
    #This line filtered the registrations by the fuel type of eletric.
    (nyc_cars['Fuel Type'] == 'ELECTRIC')
]

Next, I'm checking to see if the filtering worked before I export it to a new df. While I could filter this data myself through the OpenData portal, I prefer to do it manually in code so that I can have the full, unmodified record file in storage, plus, they're changing the portal at the end of January, and I'm not sure what that future holds.

In [ ]:
nyc_evs_filtered.head()

In [ ]:
len(nyc_evs_filtered)

All the filters appear to have worked sucessfully as this count matches what came out of the OpenData portal's filters.

Now I'm going to preview the New York state charger listings and prepare to filter.

In [ ]:
ny_chargers.head()

So it appears there's not a county column on this dataset to use for filtering like in the state registration set. To filter, as tedious as this will be, I'm going to make five lists ZIP codes, one for each county, to be able to filter to their own dataframes, add a county column and concat into a df of the city's chargers.

(ZIP codes were retriveved from https://www.zip-codes.com/ and were filtered by county. I then copied their five county tables into Google Sheets and pulled them out with the =RIGHT formula and added the ' ', with ="'" & CELL & "'," and pulled them in. [Google Sheet](https://docs.google.com/spreadsheets/d/1T9KsswpceAEB8igOB3k95JAsupEvjUtfvYjHGQ9XF8M/edit?usp=sharing)

In [ ]:
bronx_zips = [
    '10451', '10452', '10453', '10454', '10455', '10456', '10457', '10458', '10459', '10460'
    '10461', '10462', '10463', '10464', '10465', '10466', '10467', '10468', '10469', '10470'
    '10471', '10472', '10473', '10474', '10475',
]    
kings_zips = [
    '11202', '11241', '11242', '11243', '11245', '11247', '11251', '11252', '11256', '11239',
    '11232', '11231', '11222', '11249', '11217', '11228', '11224', '11237', '11205', '11225',
    '11238', '11216', '11210', '11213', '11215', '11211', '11201', '11209', '11233', '11218',
    '11203', '11223', '11229', '11204', '11235', '11221', '11230', '11206', '11212', '11234', 
    '11214', '11207', '11236', '11226', '11208', '11219', '11220',
]
newyork_zips = [
    '10001', '10002', '10003', '10004', '10005', '10006', '10007', '10008', '10009', '10010',
    '10011', '10012', '10013', '10014', '10016', '10017', '10018', '10019', '10020', '10021',
    '10022', '10023', '10024', '10025', '10026', '10027', '10028', '10029', '10030', '10031',
    '10032', '10033', '10034', '10035', '10036', '10037', '10038', '10039', '10040', '10041',
    '10043', '10044', '10045', '10055', '10060', '10065', '10069', '10075', '10080', '10081',
    '10087', '10090', '10101', '10102', '10103', '10104', '10105', '10106', '10107', '10108', 
    '10109', '10110', '10111', '10112', '10113', '10114', '10115', '10116', '10117', '10118',
    '10119', '10120', '10121', '10122', '10123', '10124', '10125', '10126', '10128', '10129',
    '10130', '10131', '10132', '10133', '10138', '10150', '10151', '10152', '10153', '10154',
    '10155', '10156', '10157', '10158', '10159', '10160', '10162', '10163', '10164', '10165', 
    '10166', '10167', '10168', '10169', '10170', '10171', '10172', '10173', '10174', '10175',
    '10176', '10177', '10178', '10179', '10185', '10199', '10203', '10211', '10212', '10213',
    '10242', '10249', '10256', '10258', '10259', '10260', '10261', '10265', '10268', '10269',
    '10270', '10271', '10272', '10273', '10274', '10275', '10276', '10277', '10278', '10279',
    '10280', '10281', '10282', '10285', '10286',
]
queens_zips = [
    '11004', '11005', '11101', '11102', '11103', '11104', '11105', '11106', '11109', '11120',
    '11351', '11352', '11354', '11355', '11356', '11357', '11358', '11359', '11360', '11361',
    '11362', '11363', '11364', '11365', '11366', '11367', '11368', '11369', '11370', '11371',
    '11372', '11373', '11374', '11375', '11377', '11378', '11379', '11380', '11385', '11386',
    '11405', '11411', '11412', '11413', '11414', '11415', '11416', '11417', '11418', '11419',
    '11420', '11421', '11422', '11423', '11424', '11425', '11426', '11427', '11428', '11429',
    '11430', '11431', '11432', '11433', '11434', '11435', '11436', '11437', '11439', '11451',
    '11499', '11690', '11691', '11692', '11693', '11694', '11695', '11697',
]
richmond_zips = [
    '10301', '10302', '10303', '10304', '10305', '10306', '10307', '10308', '10309', '10310',
    '10311', '10312', '10313', '10314'
]

Now I'm making those five dataframes! (I feel like there was probably a more streamlined way to do this, but I didn't want to mess up on this step, so I just did what I knew I could sucessfully.)

In [ ]:
bronx_chargers = pd.DataFrame(ny_chargers.loc[ny_chargers["ZIP"].isin(bronx_zips)])
kings_chargers = pd.DataFrame(ny_chargers.loc[ny_chargers["ZIP"].isin(kings_zips)])
newyork_chargers = pd.DataFrame(ny_chargers.loc[ny_chargers["ZIP"].isin(newyork_zips)])
queens_chargers = pd.DataFrame(ny_chargers.loc[ny_chargers["ZIP"].isin(queens_zips)])
richmond_chargers = pd.DataFrame(ny_chargers.loc[ny_chargers["ZIP"].isin(richmond_zips)])

Adding a county column and adding the respective county to it.

In [ ]:
bronx_chargers['County'] = "BRONX"
kings_chargers['County'] = "KINGS"
newyork_chargers['County'] = "NEW YORK"
queens_chargers['County'] = "QUEENS"
richmond_chargers['County'] = "RICHMOND"

Now I'm bring it all back together to create our dataframe of chargers in the city!

In [ ]:
nyc_chargers = pd.concat([bronx_chargers, kings_chargers, newyork_chargers, queens_chargers, richmond_chargers])

In [ ]:
evs_pivot = nyc_evs.pivot_table(index='County', values='VIN', aggfunc='count').reset_index()
evs_pivot.columns = ['County', 'Number of EVs']
print(evs_pivot)

In [ ]:
chargers_pivot = nyc_chargers.pivot_table(index='County', values =['EV Level2 EVSE Num', 'EV DC Fast Count'], aggfunc='sum', fill_value=0).reset_index()
chargers_pivot.columns = ['County', 'DC Fast', 'Level 2']
print(chargers_pivot)

Now that the data is in nice, like-formatted pivot tables, it's time to merge and plot!

In [ ]:
nyc_ev_life = pd.merge(evs_pivot, chargers_pivot, on='County', how='outer')
nyc_ev_life_df = pd.DataFrame(nyc_ev_life)
print(nyc_ev_life_df)

In [ ]:
nyc_ev_life_df.to_csv('../outputs/NYC EV Life.csv', index=True)

Setting the index as the county so that it appears as the proper label instead of 0, 1, 2...

In [ ]:
nyc_ev_life_df.set_index('County', inplace=True)

And now to plot! I chose a horizontal bar chart because it seemed the easiest to consume of the bar charts. Find more detailed descriptions of each line below.

In [ ]:
ax = nyc_ev_life_df.plot(kind='barh', figsize=(10,6), width=.8)
##Setting all my labels and making sure they don't overhang.
plt.title('EV Count vs. Charger Count by NYC County')
plt.xlabel('Count')
plt.ylabel('County')
plt.tight_layout()

##This was a bit confusing, but it displays the numbers on the graph. It sets a place for the text to go, then plots the associated integer
for p in ax.patches:
    ax.text(p.get_width() + 0.1, p.get_y() + p.get_height() /2,
        str(int(p.get_width())),
        horizontalalignment='left',
        verticalalignment='center')
##Saving...
plt.savefig('../outputs/EV Chargers vs Vehicles Graph.jpg', format='jpg', bbox_inches='tight')